# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to <b>determine which demographic groups respond best to which offer type</b>. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is <b>feeling the influence of the offer</b>for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)



**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [5]:
import pandas as pd
import numpy as np
import math
import json
# import matplotlib
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

## Exploratory Data Analysis

### portfolio.json
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)



In [8]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [2]:
# # coded the offer id
# offer_code = {'ae264e3637204a6fb9bb56bc8210ddfd' : "A", 
#               '4d5c57ea9a6940dd891ad53e9dbe8da0': "B", 
#               '3f207df678b143eea3cee63160fa8bed': "C",
#               '9b98b8c7a33c4b65b9aebfe6a799e6d9': "D",
#               '0b1e1539f2cc45b7b9fa7c272da2e1d7' : "E",
#               '2298d6c36e964ae4a3e7e9706d1fb8c2': "F"
#               ''
#              }
# portfolio["id"] = portfolio["id"].replace(offer_code)

In [9]:
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 608.0+ bytes


In [10]:
d=list({'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}.values())
d

['9b98b8c7a33c4b65b9aebfe6a799e6d9']

In [ ]:
portfolio.info()

In [11]:
portfolio.shape

(10, 6)

In [ ]:
portfolio.describe()

In [ ]:
portfolio.isnull().sum()

In [ ]:
portfolio.rename({'id': 'portfolio_id'}, axis =1, inplace=True)

In [ ]:
port_cat = ["reward", "duration", "offer_type", "difficulty"]


In [ ]:
for i in port_cat:
    print (i, '=',  portfolio[i].unique())

In [ ]:
len(portfolio['portfolio_id'].unique())


- The data types have been well coded
- The portfolio rewards are all unique

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

In [ ]:
profile.head(2)

In [ ]:
profile.info()

In [ ]:
#change datatype for became_memberon column
#first, convert col to string then df
profile['became_member_on'] = profile['became_member_on'].apply(str)


profile['became_member_on'] =  pd.to_datetime(profile['became_member_on'], infer_datetime_format=True)

profile.head(2)

In [ ]:
profile['became_member_on'].dtype

In [ ]:
profile.describe()

In [ ]:
profile.rename({'id': 'profile_id'}, axis =1, inplace=True)

In [ ]:
profile.shape

In [ ]:
print("No of unique records in the df (given by report_number) is {}".format(profile['profile_id'].
                                                                                nunique()))

In [ ]:
profile.isnull().sum()

In [ ]:
prof_cat =["gender"]

In [ ]:
profile['age'].hist()

#over 2000 profiles 118 years old is weird

In [ ]:
for i in prof_cat:
    print (i, '=',  profile[i].unique())
    

 

In [ ]:
subdf_118 = profile[['age','gender','income']][profile['age']==118]
                    
                    
print(subdf_118.head(2))                    
print(subdf_118.tail(2))


print('no of persons aged 118 is ',subdf_118.shape[0])
print('corresponding Gender nulls:', subdf_118['gender'].isnull().sum())
print('corresponding income nulls :', subdf_118['income'].isnull().sum())


In [ ]:
# specific count of profiles exactly of age = 118 and check the subset where age is 118 for the other columns
profile[profile['age']==118].count()

- some gender has been coded none and o. with missing data.
- the datatype for date is not proper.We turn to datetime
- max age 118 must be an oulier,we check

it is clear that these profiles with this age have no income nor gender recorded, Probably persons who do not like to share such information.

To confirm this, we create a sub df of only persons aged 118 and find that every person aged 118 has both gendrr and income missing.

## Transcript


In [10]:
#recode name for ids of pr
transcript['value_trans'] = transcript['value'].apply(lambda x: list(x.values())[0]) #.groupby(['person','value'])['event'].count()
transcript['value_trans'] = transcript.value_trans.replace(offer_code)
transcript.tail()


,person,event,value,time,value_trans
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714,1.59
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714,9.53
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714,3.61
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714,3.53
306533,c02b10e8752c4d8e9b73f918558531f7,transaction,{'amount': 4.05},714,4.05


In [ ]:
transcript.groupby(['person','value_trans'])['event'].count()

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [ ]:
transcript[transcript['event'] == "transaction"] 

In [14]:
transcript

,person,event,value,time,value_trans
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,D
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,E
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,B
...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714,1.59
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714,9.53
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714,3.61
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714,3.53


In [ ]:
transcript[transcript['event'] == "offer received"]

In [ ]:
#check if the offer for email and web only are also part of sales data

transcript[transcript['value'] == {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}]


In [ ]:
transcript.head(3)

In [ ]:
transcript.shape

In [ ]:
transcript.time.

In [ ]:
# check for any nulls existing in this df.
transcript.isnull().sum()

In [ ]:
# check for duplicates
transcript.columns.duplicated().sum()

In [ ]:
# checking the count of each event type
transcript["event"].value_counts()

In [ ]:
transcript.event.unique()

In [ ]:
#we try to recode the value column

In [ ]:
transcript.describe()

In [ ]:
714/24

In [ ]:
#isadir